<a href="https://colab.research.google.com/github/Auracelle/Auracelle-Charlie-App/blob/main/Copy_of_AGENTIC_AI_Auracelle_Charlie_Live_2025_11_24_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === Auracelle Charlie - Live 2025: Colab Setup (Reserved Domain) ===
# Installs, cleanup, and ngrok config for aiwargame.ngrok.app
import os

try:
    import pip
    !pip -q install --upgrade pip >/dev/null 2>&1
    !pip -q install streamlit pyngrok==7.2.1 plotly pyvis networkx geopandas folium shapely pyproj rtree >/dev/null 2>&1
except Exception as e:
    print("Dependency install encountered an issue (continuing):", e)

!pkill -f ngrok || true
!pkill -f streamlit || true

try:
    from pyngrok import ngrok
    try:
        ngrok.kill()
        ngrok.disconnect()
    except Exception:
        pass
except Exception:
    pass

os.environ["NGROK_AUTHTOKEN"] = "2vmh7uE9lpuOWrldBNSV68hJKH7_4Ukd3XG92jWofsVoZALiJ"
!ngrok config add-authtoken $NGROK_AUTHTOKEN

print("✅ Setup complete. Reserved domain: aiwargame.ngrok.app")

^C
^C
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Setup complete. Reserved domain: aiwargame.ngrok.app


In [ ]:
# ========================================
# CELL 1: Install Dependencies
# ========================================
!pip install -q pyngrok streamlit networkx matplotlib numpy pandas torch plotly wbgapi requests

# ========================================
# CELL 2: Setup Environment & AGPO Data Package
# ========================================
import os, time, subprocess, sys
from pyngrok import ngrok
from IPython.display import display, HTML

# Cleanup existing processes
!pkill -f streamlit || true
!pkill -f ngrok || true
try:
    ngrok.disconnect("http://localhost:8501")
except:
    pass
try:
    ngrok.kill()
except:
    pass

ngrok.set_auth_token("2vmh7uE9lpuOWrldBNSV68hJKH7_4Ukd3XG92jWofsVoZALiJ")
os.makedirs("pages", exist_ok=True)
os.makedirs("agpo_data", exist_ok=True)
print("✅ Setup complete")

# ========================================
# CELL 3: Create AGPO Data Package - World Bank API Integration
# ========================================
agpo_worldbank = '''"""
AGPO Data Package - World Bank API Integration
Fetches real-world economic and development indicators
"""
import wbgapi as wb
import pandas as pd
import streamlit as st

@st.cache_data(ttl=3600)
def get_world_bank_indicator(indicator_code, country_codes, start_year=2015, end_year=2024):
    """
    Fetch World Bank indicator data for specified countries

    Args:
        indicator_code: WB indicator (e.g., 'NY.GDP.MKTP.CD' for GDP)
        country_codes: List of ISO3 country codes (e.g., ['USA', 'CHN'])
        start_year: Start year for data
        end_year: End year for data

    Returns:
        DataFrame with country, year, indicator value
    """
    try:
        data = wb.data.DataFrame(
            indicator_code,
            country_codes,
            time=range(start_year, end_year + 1),
            labels=True
        )

        # Reshape data
        data_reset = data.reset_index()
        data_melted = data_reset.melt(
            id_vars=['Country'],
            var_name='Year',
            value_name='Value'
        )
        data_melted['Indicator'] = indicator_code

        return data_melted
    except Exception as e:
        st.warning(f"World Bank API error: {e}")
        return pd.DataFrame()

@st.cache_data(ttl=3600)
def get_many_indicators(indicator_codes, country_codes=['USA', 'CHN', 'GBR', 'JPN', 'IND', 'BRA', 'ARE'], start_year=2015, end_year=2024):
    """
    Fetch multiple World Bank indicators at once

    Common indicators:
    - NY.GDP.MKTP.CD: GDP (current US$)
    - MS.MIL.XPND.GD.ZS: Military expenditure (% of GDP)
    - IT.NET.USER.ZS: Internet users (% of population)
    - SP.POP.TOTL: Total population
    - NY.GDP.PCAP.CD: GDP per capita
    """
    all_data = []
    for indicator in indicator_codes:
        df = get_world_bank_indicator(indicator, country_codes, start_year, end_year)
        if not df.empty:
            all_data.append(df)

    if all_data:
        return pd.concat(all_data, ignore_index=True)
    return pd.DataFrame()

def get_latest_gdp(country_code):
    """Get most recent GDP data for a country"""
    try:
        data = get_world_bank_indicator('NY.GDP.MKTP.CD', [country_code], 2020, 2024)
        if not data.empty:
            latest = data.dropna().sort_values('Year', ascending=False).iloc[0]
            return latest['Value'] / 1e12  # Convert to trillions
        return None
    except:
        return None

def get_latest_military_expenditure(country_code):
    """Get most recent military expenditure as % of GDP"""
    try:
        data = get_world_bank_indicator('MS.MIL.XPND.GD.ZS', [country_code], 2020, 2024)
        if not data.empty:
            latest = data.dropna().sort_values('Year', ascending=False).iloc[0]
            return latest['Value']
        return None
    except:
        return None

def get_internet_penetration(country_code):
    """Get internet users as % of population"""
    try:
        data = get_world_bank_indicator('IT.NET.USER.ZS', [country_code], 2020, 2024)
        if not data.empty:
            latest = data.dropna().sort_values('Year', ascending=False).iloc[0]
            return latest['Value']
        return None
    except:
        return None
'''

with open('agpo_data/worldbank.py', 'w') as f:
    f.write(agpo_worldbank)

# ========================================
# CELL 4: Create AGPO Data Package - Export Controls API
# ========================================
agpo_exportcontrol = '''"""
AGPO Data Package - US Export Controls Integration
Consolidated Screening List (CSL) from trade.gov
"""
import requests
import pandas as pd
import streamlit as st
from datetime import datetime

@st.cache_data(ttl=86400)  # Cache for 24 hours
def fetch_consolidated_screening_list():
    """
    Fetch US Consolidated Screening List (Entity List, DPL, UVL, etc.)
    Returns DataFrame with sanctioned entities
    """
    url = "https://api.trade.gov/consolidated_screening_list/search"

    try:
        params = {
            "size": 100,
            "offset": 0
        }

        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()

        data = response.json()
        results = data.get('results', [])

        # Parse results
        records = []
        for item in results:
            records.append({
                'name': item.get('name', 'N/A'),
                'country': item.get('addresses', [{}])[0].get('country', 'N/A') if item.get('addresses') else 'N/A',
                'source': item.get('source', 'N/A'),
                'type': item.get('type', 'N/A'),
                'programs': ', '.join(item.get('programs', [])),
                'remarks': item.get('remarks', '')
            })

        return pd.DataFrame(records)

    except Exception as e:
        st.warning(f"Export Controls API error: {e}")
        return pd.DataFrame()

def check_entity_sanctions(entity_name):
    """Check if an entity appears on screening lists"""
    df = fetch_consolidated_screening_list()
    if df.empty:
        return None

    matches = df[df['name'].str.contains(entity_name, case=False, na=False)]
    return matches

def get_sanctioned_countries():
    """Get list of countries with sanctioned entities"""
    df = fetch_consolidated_screening_list()
    if df.empty:
        return []

    return df['country'].value_counts().to_dict()
'''

with open('agpo_data/exportcontrol.py', 'w') as f:
    f.write(agpo_exportcontrol)

# ========================================
# CELL 5: Create AGPO Data Package - SIPRI Integration
# ========================================
agpo_sipri = '''"""
AGPO Data Package - SIPRI Military Expenditure Integration
Note: SIPRI requires manual CSV download. This module processes the data.
"""
import pandas as pd
import streamlit as st
import io

def parse_sipri_csv(uploaded_file):
    """
    Parse SIPRI military expenditure CSV
    Download from: https://www.sipri.org/databases/milex
    """
    try:
        df = pd.read_csv(uploaded_file, encoding='utf-8')
        return df
    except Exception as e:
        st.error(f"SIPRI CSV parsing error: {e}")
        return pd.DataFrame()

def get_military_expenditure_by_country(df, country_name, year=2023):
    """Extract military expenditure for specific country and year"""
    try:
        if 'Country' in df.columns and str(year) in df.columns:
            country_data = df[df['Country'] == country_name]
            if not country_data.empty:
                value = country_data[str(year)].iloc[0]
                return value
        return None
    except:
        return None

def get_top_military_spenders(df, year=2023, top_n=10):
    """Get top N military spenders for a given year"""
    try:
        if str(year) in df.columns:
            top = df.nlargest(top_n, str(year))[['Country', str(year)]]
            return top
        return pd.DataFrame()
    except:
        return pd.DataFrame()
'''

with open('agpo_data/sipri.py', 'w') as f:
    f.write(agpo_sipri)

# ========================================
# CELL 6: Create AGPO Package Init
# ========================================
agpo_init = '''"""
AGPO Data Package - AI Governance Policy Optimization
Real-world data integration for wargaming simulations
"""
from .worldbank import *
from .exportcontrol import *
from .sipri import *

__version__ = "1.0.0"
'''

with open('agpo_data/__init__.py', 'w') as f:
    f.write(agpo_init)

print("✅ AGPO Data Package created")

# ========================================
# CELL 7: Create Adjudicator Module with API Integration
# ========================================
adjudicator_code = '''import random
import numpy as np
from datetime import datetime
import sys
sys.path.insert(0, ".")

class AgenticAdjudicator:
    """AI Agentic Adjudicator - Neutral referee with real-world data integration"""

    def __init__(self, mode="neutral"):
        self.mode = mode
        self.event_history = []
        self.tension_index = 0.5
        self.shock_types = [
            "economic_sanctions", "cyber_attack", "public_protest",
            "un_resolution", "trade_disruption", "diplomatic_incident",
            "tech_breakthrough", "alliance_shift", "intel_leak"
        ]
        self.real_world_data = {}

    def integrate_real_world_data(self, country_code, gdp=None, mil_exp=None, internet=None, sanctions=None):
        """Store real-world data for adjudication calculations"""
        self.real_world_data[country_code] = {
            'gdp': gdp,
            'military_expenditure_pct': mil_exp,
            'internet_penetration': internet,
            'sanctioned_entities': sanctions
        }

    def calculate_tension_index(self, actor_positions, power_levels, alignment_graph):
        """Calculate geopolitical tension with real-world data weighting"""
        position_divergence = np.std([hash(p) % 100 for p in actor_positions.values()]) / 100
        power_imbalance = np.std(list(power_levels.values()))
        alignment_factor = 1.0 - (sum(alignment_graph.values()) / (len(alignment_graph) + 1e-6))

        # Add real-world military expenditure factor
        mil_exp_factor = 0.0
        if self.real_world_data:
            mil_exps = [d.get('military_expenditure_pct', 2.0) for d in self.real_world_data.values() if d.get('military_expenditure_pct')]
            if mil_exps:
                mil_exp_factor = (np.mean(mil_exps) - 2.0) / 10.0  # Normalize around 2% baseline

        tension = (position_divergence * 0.3 + power_imbalance * 0.2 + alignment_factor * 0.3 + abs(mil_exp_factor) * 0.2)
        self.tension_index = max(0.0, min(1.0, tension))
        return self.tension_index

    def detect_deception(self, stated_position, historical_actions, power_level):
        """Detect potential deception in actor statements"""
        deception_score = 0.0

        if len(historical_actions) > 0:
            consistency = sum([1 for a in historical_actions if a == stated_position]) / len(historical_actions)
            deception_score += (1.0 - consistency) * 0.5

        if power_level < 0.5 and "aggressive" in stated_position.lower():
            deception_score += 0.3

        return min(1.0, deception_score)

    def inject_shock(self, current_round, tension_level):
        """Inject external shock event based on tension and real-world factors"""
        # Increase shock probability if sanctions are detected
        sanctions_multiplier = 1.0
        if self.real_world_data:
            total_sanctions = sum([d.get('sanctioned_entities', 0) for d in self.real_world_data.values()])
            if total_sanctions > 0:
                sanctions_multiplier = 1.3

        shock_probability = tension_level * 0.3 * sanctions_multiplier

        if random.random() < shock_probability:
            shock_type = random.choice(self.shock_types)
            impact_magnitude = random.uniform(0.1, 0.5) * tension_level

            event = {
                "round": current_round,
                "type": shock_type,
                "magnitude": impact_magnitude,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "narrative": self._generate_narrative(shock_type, impact_magnitude),
                "real_world_triggered": sanctions_multiplier > 1.0
            }

            self.event_history.append(event)
            return event

        return None

    def _generate_narrative(self, shock_type, magnitude):
        """Generate narrative text for shock events"""
        severity = "severe" if magnitude > 0.3 else "moderate"
        narratives = {
            "economic_sanctions": f"Breaking: Coalition imposes {severity} economic sanctions",
            "cyber_attack": f"Alert: {severity.capitalize()} cyber incident targets critical infrastructure",
            "public_protest": f"Developing: {severity.capitalize()} public demonstrations challenge policy",
            "un_resolution": f"UN Security Council debates {severity} resolution",
            "trade_disruption": f"Economic shock: {severity.capitalize()} trade route disruption",
            "diplomatic_incident": f"Diplomatic crisis: {severity.capitalize()} incident strains relations",
            "tech_breakthrough": f"Tech update: {severity.capitalize()} AI capability announced",
            "alliance_shift": f"Geopolitical shift: {severity.capitalize()} realignment detected",
            "intel_leak": f"Intelligence alert: {severity.capitalize()} data leak exposed"
        }
        return narratives.get(shock_type, "Unknown event occurred")

    def adjudicate(self, actor_beliefs, power_levels, alignment_graph, current_round):
        """Main adjudication function with real-world data integration"""
        tension = self.calculate_tension_index(actor_beliefs, power_levels, alignment_graph)
        shock_event = self.inject_shock(current_round, tension)

        confidence = 1.0 - (tension * 0.3)

        deception_scores = {}
        for actor, belief in actor_beliefs.items():
            hist = [e.get(actor) for e in self.event_history if actor in e]
            deception_scores[actor] = self.detect_deception(belief, hist, power_levels.get(actor, 0.5))

        return {
            "tension_index": tension,
            "shock_event": shock_event,
            "confidence_score": confidence,
            "deception_scores": deception_scores,
            "narrative": shock_event["narrative"] if shock_event else "Situation stable",
            "round": current_round,
            "real_world_integrated": len(self.real_world_data) > 0
        }
'''

with open('adjudicator.py', 'w') as f:
    f.write(adjudicator_code)

print("✅ Enhanced Adjudicator with API integration created")

# ========================================
# CELL 8: Create Login Page
# ========================================
app_code = '''import streamlit as st

st.set_page_config(page_title="Auracelle Charlie Phase 2", layout="wide", initial_sidebar_state="collapsed")
st.title("🔐 Auracelle Charlie Phase 2: AI Adjudicator + Real-World Data")

if "authenticated" not in st.session_state:
    st.session_state["authenticated"] = False

with st.form("login_form"):
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")

    st.markdown("### 🎮 Phase 2 Features")
    st.info(
        "**New Capabilities:**\\n"
        "- 🤖 AI Agentic Adjudicator (neutral referee)\\n"
        "- 🌍 World Bank API (GDP, military expenditure, internet penetration)\\n"
        "- 🚫 US Export Controls API (sanctions screening)\\n"
        "- 💥 External shock injection system\\n"
        "- 🎭 Deception detection with real-world data\\n"
        "- 📊 Real-time geopolitical tension tracking\\n"
        "- 📜 Complete event history log"
    )

    submit = st.form_submit_button("Login")

if submit:
    if password == "charlie2025":
        st.session_state["authenticated"] = True
        st.session_state["username"] = username
        st.switch_page("pages/simulation.py")
    else:
        st.error("Incorrect password. Access denied.")

if st.session_state.get("authenticated", False):
    st.switch_page("pages/simulation.py")
'''

with open('app.py', 'w') as f:
    f.write(app_code)

print("✅ Login page created")

# ========================================
# CELL 9: Create Simulation Page with Full API Integration
# ========================================
sim_code = '''import streamlit as st
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import sys
sys.path.insert(0, ".")
from adjudicator import AgenticAdjudicator

# Import AGPO data modules
try:
    from agpo_data.worldbank import get_latest_gdp, get_latest_military_expenditure, get_internet_penetration, get_many_indicators
    from agpo_data.exportcontrol import fetch_consolidated_screening_list, get_sanctioned_countries
    from agpo_data.sipri import parse_sipri_csv
    AGPO_AVAILABLE = True
except ImportError as e:
    st.warning(f"AGPO modules not fully loaded: {e}")
    AGPO_AVAILABLE = False

st.set_page_config(page_title="Auracelle Charlie Phase 2", layout="wide", initial_sidebar_state="collapsed")

if not st.session_state.get("authenticated", False):
    st.warning("Please log in first.")
    st.switch_page("app.py")

st.title("🎯 AI Governance Policy Wargame with Real-World Data")
st.header("Auracelle Charlie Phase 2: Intelligence + Adjudication + APIs 🚀")

# Initialize session state
if "round" not in st.session_state:
    st.session_state["round"] = 1
if "q_table" not in st.session_state:
    st.session_state["q_table"] = {}
if "adjudicator" not in st.session_state:
    st.session_state["adjudicator"] = AgenticAdjudicator(mode="neutral")
if "event_log" not in st.session_state:
    st.session_state["event_log"] = []
# New: round-level metrics and agent controls
if "round_metrics_trace" not in st.session_state:
    st.session_state["round_metrics_trace"] = []
if "episode_length" not in st.session_state:
    st.session_state["episode_length"] = 5
if "stochastic_exploration" not in st.session_state:
    st.session_state["stochastic_exploration"] = False
if "api_data_loaded" not in st.session_state:
    st.session_state["api_data_loaded"] = False

adjudicator = st.session_state["adjudicator"]

# Sidebar for API Data Controls
with st.sidebar:
    st.header("🌍 Real-World Data Integration")

    if st.button("🔄 Refresh API Data"):
        st.session_state["api_data_loaded"] = False
        st.rerun()

    st.markdown("---")
    st.markdown("**Data Sources:**")
    st.markdown("- World Bank API ✅")
    st.markdown("- US Export Controls ✅")
    st.markdown("- SIPRI (CSV Upload)")

    sipri_file = st.file_uploader("Upload SIPRI CSV", type=['csv'])

with st.expander("📘 Phase 2 Instructions", expanded=False):
    st.markdown("""
    **Real-World Data Integration:**

    1. **World Bank API**: Automatically fetches GDP, military expenditure, internet penetration
    2. **Export Controls**: Checks US sanctions lists for each country
    3. **SIPRI Data**: Upload military expenditure CSV for enhanced analysis
    4. **AI Adjudicator**: Uses real data to inform tension calculations and shock probabilities
    5. **Interactive Feedback**: Policy decisions are evaluated against real-world economic/military data
    """)

# Country mapping to ISO codes
country_to_iso = {
    "Dubai": "ARE",
    "United Kingdom": "GBR",
    "United States": "USA",
    "Japan": "JPN",
    "China": "CHN",
    "Brazil": "BRA",
    "India": "IND"
}

policy_options = [
    "Interim Final Rule on Artificial Intelligence Diffusion",
    "Data Privacy Act 118th",
    "The European Union AI Act",
    "NATO Article 5"
]
selected_policy = st.selectbox("Select Policy Scenario", policy_options)

country_options = ["Dubai", "United Kingdom", "United States", "Japan", "China", "Brazil", "India", "NATO"]

# Default data structure
default_data = {
    "Dubai": {"gdp": 0.5, "influence": 0.7, "position": "Moderate regulatory stance", "mil_exp": 5.6, "internet": 99.0},
    "United Kingdom": {"gdp": 3.2, "influence": 0.85, "position": "Supports EU-style data protection", "mil_exp": 2.2, "internet": 96.0},
    "United States": {"gdp": 21.0, "influence": 0.95, "position": "Favors innovation over regulation", "mil_exp": 3.4, "internet": 92.0},
    "Japan": {"gdp": 5.1, "influence": 0.88, "position": "Pro-regulation for trust", "mil_exp": 1.0, "internet": 95.0},
    "China": {"gdp": 17.7, "influence": 0.93, "position": "Strict state-driven AI governance", "mil_exp": 1.7, "internet": 73.0},
    "Brazil": {"gdp": 2.0, "influence": 0.75, "position": "Leaning toward EU-style regulation", "mil_exp": 1.4, "internet": 81.0},
    "India": {"gdp": 3.7, "influence": 0.82, "position": "Strategic tech balancing", "mil_exp": 2.4, "internet": 43.0},
    "NATO": {"gdp": 25.0, "influence": 0.97, "position": "Collective security & data interoperability", "mil_exp": 2.5, "internet": 90.0}
}

# Load real-world data
if AGPO_AVAILABLE and not st.session_state["api_data_loaded"]:
    with st.spinner("🌍 Loading real-world data from APIs..."):
        for country, iso_code in country_to_iso.items():
            try:
                # Get World Bank data
                gdp = get_latest_gdp(iso_code)
                mil_exp = get_latest_military_expenditure(iso_code)
                internet = get_internet_penetration(iso_code)

                # Update default data with real values
                if gdp is not None:
                    default_data[country]["gdp"] = round(gdp, 2)
                if mil_exp is not None:
                    default_data[country]["mil_exp"] = round(mil_exp, 2)
                if internet is not None:
                    default_data[country]["internet"] = round(internet, 1)

                # Integrate into adjudicator
                adjudicator.integrate_real_world_data(
                    iso_code,
                    gdp=default_data[country]["gdp"],
                    mil_exp=default_data[country]["mil_exp"],
                    internet=default_data[country]["internet"]
                )

            except Exception as e:
                st.warning(f"Could not load data for {country}: {e}")

        # Get sanctions data
        try:
            sanctions_df = fetch_consolidated_screening_list()
            if not sanctions_df.empty:
                sanctioned_countries = get_sanctioned_countries()
                for country, iso_code in country_to_iso.items():
                    country_name = country if country != "Dubai" else "United Arab Emirates"
                    sanction_count = sanctioned_countries.get(country_name, 0)
                    adjudicator.integrate_real_world_data(
                        iso_code,
                        sanctions=sanction_count
                    )
        except:
            pass

        st.session_state["api_data_loaded"] = True
        st.success("✅ Real-world data loaded successfully!")

selected_country_a = st.selectbox("Select Country A", country_options, index=0)
selected_country_b = st.selectbox("Select Country B", country_options, index=1)

role_tags = ["Governance","MilitaryAI","DataPrivacy","ExportControl","Diplomacy","StandardSetting","Surveillance","Trade","TechAlliance"]
role_country_a = st.selectbox(f"Role for {selected_country_a}", role_tags, key="role_a")
role_country_b = st.selectbox(f"Role for {selected_country_b}", role_tags, key="role_b")

player_country = st.selectbox("🎖️ You represent:", country_options, index=0)

# Display real-world data comparison
st.subheader("🆚 Policy Position Comparison (Real-World Data)")
comparison_df = pd.DataFrame({
    "Metric": ["GDP (Trillion USD)", "Military Exp (% GDP)", "Internet Penetration (%)", "Influence Score", "AI Policy Position"],
    selected_country_a: [
        f"${default_data[selected_country_a]['gdp']:.2f}T",
        f"{default_data[selected_country_a]['mil_exp']:.1f}%",
        f"{default_data[selected_country_a]['internet']:.1f}%",
        default_data[selected_country_a]["influence"],
        default_data[selected_country_a]["position"]
    ],
    selected_country_b: [
        f"${default_data[selected_country_b]['gdp']:.2f}T",
        f"{default_data[selected_country_b]['mil_exp']:.1f}%",
        f"{default_data[selected_country_b]['internet']:.1f}%",
        default_data[selected_country_b]["influence"],
        default_data[selected_country_b]["position"]
    ]
})
st.table(comparison_df)

player_new_position = st.text_input("📜 Propose New Policy Position", value=default_data[player_country]["position"])
opponent_country = selected_country_b if player_country == selected_country_a else selected_country_a
alignment_score = 1.0 if player_new_position == default_data[opponent_country]["position"] else 0.0

st.markdown("---")
st.subheader("🤖 AI Agentic Adjudicator Status")

actor_beliefs = {
    selected_country_a: default_data[selected_country_a]["position"],
    selected_country_b: default_data[selected_country_b]["position"]
}
power_levels = {
    selected_country_a: default_data[selected_country_a]["influence"],
    selected_country_b: default_data[selected_country_b]["influence"]
}
alignment_graph = {(selected_country_a, selected_country_b): alignment_score}

adjudication = adjudicator.adjudicate(actor_beliefs, power_levels, alignment_graph, st.session_state["round"])

col1, col2, col3, col4 = st.columns(4)
with col1:
    tension_color = "🔴" if adjudication["tension_index"] > 0.7 else ("🟡" if adjudication["tension_index"] > 0.4 else "🟢")
    st.metric(f"{tension_color} Tension", f"{adjudication['tension_index']*100:.1f}%")
with col2:
    st.metric("🎲 Confidence", f"{adjudication['confidence_score']*100:.1f}%")
with col3:
    st.metric("📊 Alignment", f"{alignment_score*100:.0f}%")
with col4:
    real_world_icon = "✅" if adjudication.get("real_world_integrated") else "⚠️"
    st.metric(f"{real_world_icon} Real Data", "Active" if adjudication.get("real_world_integrated") else "Simulated")

st.markdown("### 🎭 Deception Detection")
deception_df = pd.DataFrame([
    {"Actor": k, "Risk": f"{v*100:.1f}%", "Status": "⚠️ Suspicious" if v > 0.5 else "✅ Consistent"}
    for k, v in adjudication["deception_scores"].items()
])
st.dataframe(deception_df, use_container_width=True)

if adjudication["shock_event"]:
    shock_prefix = "💥 **REAL-WORLD TRIGGERED SHOCK**" if adjudication["shock_event"].get("real_world_triggered") else "💥 **EXTERNAL SHOCK**"
    st.warning(f"{shock_prefix}\\n\\n{adjudication['narrative']}")
    st.session_state["event_log"].append(adjudication["shock_event"])
else:
    st.info(f"ℹ️ {adjudication['narrative']}")

# --- Round header + Agent-style controls ---
round_col1, round_col2, round_col3 = st.columns([1, 1, 1])

with round_col1:
    st.subheader(f"🕐 Round: {st.session_state['round']}")

with round_col2:
    # Episode length controls how many rounds conceptually form one episode
    st.session_state["episode_length"] = st.slider(
        "Episode Length (rounds)",
        min_value=1,
        max_value=30,
        value=st.session_state.get("episode_length", 5),
        key="episode_length_slider"
    )

with round_col3:
    st.session_state["stochastic_exploration"] = st.toggle(
        "Enable Stochastic Exploration",
        value=st.session_state.get("stochastic_exploration", False),
        help="When enabled, the adjudication introduces variability into reward and risk.",
        key="stochastic_exploration_toggle"
    )

# --- Reward & Risk computation (lightweight, sim-style) ---
base_reward = (alignment_score * 0.5
               + (1.0 - adjudication["tension_index"]) * 0.3
               + adjudication["confidence_score"] * 0.2)

if st.session_state.get("stochastic_exploration", False):
    exploration_noise = np.random.normal(0, 0.05)
    base_reward = max(0.0, min(1.0, base_reward + exploration_noise))

reward = float(base_reward)
risk = float(adjudication["tension_index"])

# Log round metrics for batch evaluation
st.session_state["round_metrics_trace"].append({
    "round": int(st.session_state["round"]),
    "reward": reward,
    "risk": risk,
    "tension": float(adjudication["tension_index"]),
    "confidence": float(adjudication["confidence_score"]),
    "alignment": float(alignment_score),
})

# --- Reward & Risk display ---
metric_col1, metric_col2, metric_col3 = st.columns(3)
with metric_col1:
    st.metric("🏆 Reward", f"{reward*100:.1f}")
with metric_col2:
    st.metric("⚠️ Risk", f"{risk*100:.1f}")
with metric_col3:
    # Episode progress within the chosen episode length
    ep_pos = (st.session_state["round"] - 1) % st.session_state["episode_length"] + 1
    st.metric("📏 Episode Progress", f"{ep_pos}/{st.session_state['episode_length']}")

# --- Round navigation controls ---
nav_col1, nav_col2 = st.columns(2)
with nav_col1:
    if st.button("▶️ Next Round"):
        st.session_state["round"] += 1
        st.rerun()
with nav_col2:
    if st.button("🔄 Reset Episode"):
        st.session_state["round"] = 1
        st.session_state["round_metrics_trace"] = []
        st.rerun()

# Real-World Data Dashboard
st.markdown("---")
st.subheader("🌍 Real-World Data Dashboard")

tab1, tab2, tab3, tab4 = st.tabs(["📊 Economic Indicators", "🚫 Export Controls", "📜 Event History", "🧪 Batch Evaluation"])

with tab1:
    if AGPO_AVAILABLE:
        st.markdown("#### GDP & Military Expenditure Comparison")

        # Create comparison chart
        chart_data = []
        for country in [selected_country_a, selected_country_b]:
            if country in default_data and country != "NATO":
                chart_data.append({
                    "Country": country,
                    "GDP (Trillion USD)": default_data[country]["gdp"],
                    "Military Exp (% GDP)": default_data[country]["mil_exp"],
                    "Internet Penetration (%)": default_data[country]["internet"]
                })

        if chart_data:
            chart_df = pd.DataFrame(chart_data)

            col_a, col_b = st.columns(2)
            with col_a:
                fig_gdp = px.bar(chart_df, x="Country", y="GDP (Trillion USD)",
                                title="GDP Comparison", color="Country")
                st.plotly_chart(fig_gdp, use_container_width=True)

            with col_b:
                fig_mil = px.bar(chart_df, x="Country", y="Military Exp (% GDP)",
                               title="Military Expenditure (% of GDP)", color="Country")
                st.plotly_chart(fig_mil, use_container_width=True)

            st.markdown("#### Internet Penetration")
            fig_internet = px.bar(chart_df, x="Country", y="Internet Penetration (%)",
                                 title="Digital Infrastructure Readiness", color="Country")
            st.plotly_chart(fig_internet, use_container_width=True)
    else:
        st.warning("AGPO modules not available. Install wbgapi: pip install wbgapi")

with tab2:
    if AGPO_AVAILABLE:
        st.markdown("#### US Consolidated Screening List")
        try:
            sanctions_df = fetch_consolidated_screening_list()
            if not sanctions_df.empty:
                st.dataframe(sanctions_df.head(20), use_container_width=True)

                # Show sanctions by country
                sanctions_by_country = sanctions_df['country'].value_counts().head(10)
                fig_sanctions = px.bar(
                    x=sanctions_by_country.index,
                    y=sanctions_by_country.values,
                    labels={'x': 'Country', 'y': 'Number of Sanctioned Entities'},
                    title="Top 10 Countries with Sanctioned Entities"
                )
                st.plotly_chart(fig_sanctions, use_container_width=True)
            else:
                st.info("No sanctions data available")
        except Exception as e:
            st.error(f"Could not load sanctions data: {e}")
    else:
        st.warning("Export Controls API not available")

with tab3:
    st.markdown("#### Simulation Event Log")
    if st.session_state["event_log"]:
        for idx, event in enumerate(reversed(st.session_state["event_log"])):
            with st.expander(f"Round {event['round']} - {event['type'].replace('_', ' ').title()}", expanded=(idx==0)):
                st.markdown(f"**Timestamp:** {event['timestamp']}")
                st.markdown(f"**Narrative:** {event['narrative']}")
                st.markdown(f"**Magnitude:** {event['magnitude']:.2f}")
                st.markdown(f"**Real-World Triggered:** {'Yes ✅' if event.get('real_world_triggered') else 'No'}")
    else:
        st.info("No events yet. Events will appear as rounds progress.")

with tab4:
    st.markdown("#### 🧪 Batch Evaluation Across Rounds")

    if st.session_state["round_metrics_trace"]:
        metrics_df = pd.DataFrame(st.session_state["round_metrics_trace"])

        st.markdown("**Per-Round Metrics**")
        st.dataframe(metrics_df, use_container_width=True)

        avg_reward = metrics_df["reward"].mean()
        avg_risk = metrics_df["risk"].mean()
        avg_tension = metrics_df["tension"].mean()
        avg_confidence = metrics_df["confidence"].mean()
        avg_alignment = metrics_df["alignment"].mean()

        col_be1, col_be2, col_be3, col_be4, col_be5 = st.columns(5)
        with col_be1:
            st.metric("Avg Reward", f"{avg_reward*100:.1f}")
        with col_be2:
            st.metric("Avg Risk", f"{avg_risk*100:.1f}")
        with col_be3:
            st.metric("Avg Tension", f"{avg_tension*100:.1f}")
        with col_be4:
            st.metric("Avg Confidence", f"{avg_confidence*100:.1f}")
        with col_be5:
            st.metric("Avg Alignment", f"{avg_alignment*100:.1f}")

        st.markdown("You can use these aggregates to calibrate how stable or volatile a given policy configuration is across rounds.")
    else:
        st.info("Run a few rounds first to populate batch evaluation metrics.")


# Influence Network Visualization
st.markdown("---")
st.subheader("🕸️ Influence Network Graph")

label_a = f"{selected_country_a}\\nGDP: ${default_data[selected_country_a]['gdp']:.1f}T\\nMil: {default_data[selected_country_a]['mil_exp']:.1f}%"
label_b = f"{selected_country_b}\\nGDP: ${default_data[selected_country_b]['gdp']:.1f}T\\nMil: {default_data[selected_country_b]['mil_exp']:.1f}%"

G = nx.DiGraph()
G.add_node(label_a,
          gdp=default_data[selected_country_a]["gdp"],
          influence=default_data[selected_country_a]["influence"],
          mil_exp=default_data[selected_country_a]["mil_exp"])
G.add_node(label_b,
          gdp=default_data[selected_country_b]["gdp"],
          influence=default_data[selected_country_b]["influence"],
          mil_exp=default_data[selected_country_b]["mil_exp"])
G.add_edge(label_a, label_b, weight=alignment_score)

fig, ax = plt.subplots(figsize=(12, 8))
pos = nx.spring_layout(G, seed=42, k=2)

# Color nodes by influence
node_colors = []
for node in G.nodes():
    influence = G.nodes[node]["influence"]
    if influence >= 0.9:
        node_colors.append("darkred")
    elif influence >= 0.8:
        node_colors.append("orange")
    else:
        node_colors.append("lightblue")

nx.draw(G, pos, with_labels=True, node_color=node_colors,
        node_size=5000, font_size=8, font_weight="bold",
        edge_color="gray", arrows=True, ax=ax)

edge_labels = {(label_a, label_b): f"Alignment: {alignment_score:.2f}"}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color="purple", ax=ax)

plt.title("AI Governance Influence Network (Real-World Data)", fontsize=14, fontweight="bold")
st.pyplot(fig)

# PageRank Analysis
pagerank_scores = nx.pagerank(G, personalization={
    label_a: default_data[selected_country_a]["influence"],
    label_b: default_data[selected_country_b]["influence"]
})

st.markdown("#### 📊 Influence Propagation Analysis (PageRank)")
pagerank_df = pd.DataFrame({
    "Country": [selected_country_a, selected_country_b],
    "PageRank Score": [pagerank_scores[label_a], pagerank_scores[label_b]],
    "GDP (Trillion)": [default_data[selected_country_a]["gdp"], default_data[selected_country_b]["gdp"]],
    "Military Exp (%)": [default_data[selected_country_a]["mil_exp"], default_data[selected_country_b]["mil_exp"]]
})
st.dataframe(pagerank_df, use_container_width=True)

# Strategic Analysis
st.markdown("---")
st.subheader("🎯 Strategic Analysis & Recommendations")

col_left, col_right = st.columns(2)

with col_left:
    st.markdown(f"**{selected_country_a} Assessment**")

    # Economic power analysis
    gdp_a = default_data[selected_country_a]["gdp"]
    gdp_b = default_data[selected_country_b]["gdp"]
    gdp_ratio = gdp_a / (gdp_b + 0.01)

    if gdp_ratio > 2.0:
        st.success(f"✅ Strong economic advantage ({gdp_ratio:.1f}x GDP)")
    elif gdp_ratio < 0.5:
        st.warning(f"⚠️ Economic disadvantage ({gdp_ratio:.1f}x GDP)")
    else:
        st.info(f"📊 Comparable economic power ({gdp_ratio:.1f}x GDP)")

    # Military analysis
    mil_a = default_data[selected_country_a]["mil_exp"]
    if mil_a > 3.0:
        st.warning(f"⚠️ High military expenditure ({mil_a:.1f}% GDP) - potential aggression signal")
    elif mil_a < 1.5:
        st.info(f"🕊️ Low military expenditure ({mil_a:.1f}% GDP) - peaceful stance")
    else:
        st.success(f"✅ Moderate military spending ({mil_a:.1f}% GDP)")

    # Digital infrastructure
    internet_a = default_data[selected_country_a]["internet"]
    if internet_a > 90:
        st.success(f"✅ Advanced digital infrastructure ({internet_a:.0f}% penetration)")
    elif internet_a < 60:
        st.warning(f"⚠️ Limited digital infrastructure ({internet_a:.0f}% penetration)")
    else:
        st.info(f"📊 Developing digital infrastructure ({internet_a:.0f}% penetration)")

with col_right:
    st.markdown(f"**{selected_country_b} Assessment**")

    # Economic power analysis
    if gdp_ratio < 0.5:
        st.success(f"✅ Strong economic advantage ({1/gdp_ratio:.1f}x GDP)")
    elif gdp_ratio > 2.0:
        st.warning(f"⚠️ Economic disadvantage ({1/gdp_ratio:.1f}x GDP)")
    else:
        st.info(f"📊 Comparable economic power ({1/gdp_ratio:.1f}x GDP)")

    # Military analysis
    mil_b = default_data[selected_country_b]["mil_exp"]
    if mil_b > 3.0:
        st.warning(f"⚠️ High military expenditure ({mil_b:.1f}% GDP) - potential aggression signal")
    elif mil_b < 1.5:
        st.info(f"🕊️ Low military expenditure ({mil_b:.1f}% GDP) - peaceful stance")
    else:
        st.success(f"✅ Moderate military spending ({mil_b:.1f}% GDP)")

    # Digital infrastructure
    internet_b = default_data[selected_country_b]["internet"]
    if internet_b > 90:
        st.success(f"✅ Advanced digital infrastructure ({internet_b:.0f}% penetration)")
    elif internet_b < 60:
        st.warning(f"⚠️ Limited digital infrastructure ({internet_b:.0f}% penetration)")
    else:
        st.info(f"📊 Developing digital infrastructure ({internet_b:.0f}% penetration)")

# Adjudicator Recommendations
st.markdown("#### 🤖 Adjudicator Recommendations")

if adjudication["tension_index"] > 0.7:
    st.error("""
    **🔴 CRITICAL TENSION LEVEL**
    - Immediate de-escalation measures recommended
    - Consider confidence-building measures
    - Establish backchannel communications
    - Risk of shock events: HIGH
    """)
elif adjudication["tension_index"] > 0.4:
    st.warning("""
    **🟡 ELEVATED TENSION**
    - Monitor situation closely
    - Prepare contingency plans
    - Diplomatic engagement advised
    - Risk of shock events: MODERATE
    """)
else:
    st.success("""
    **🟢 LOW TENSION**
    - Favorable conditions for negotiation
    - Opportunity for comprehensive agreements
    - Continue current engagement strategy
    - Risk of shock events: LOW
    """)

# Export Functionality
st.markdown("---")
st.subheader("📥 Export Simulation Data")

col_export1, col_export2 = st.columns(2)

with col_export1:
    if st.button("📊 Export Full Report"):
        data_status = 'Active ✅' if adjudication.get('real_world_integrated') else 'Simulated ⚠️'

        report = f"""# Auracelle Charlie Phase 2 - Simulation Report
## Generated: {adjudication.get('timestamp', 'N/A')}
### Round: {st.session_state['round']}

## Configuration
- **Policy Scenario:** {selected_policy}
- **Country A:** {selected_country_a} (Role: {role_country_a})
- **Country B:** {selected_country_b} (Role: {role_country_b})
- **Player:** {player_country}

## Real-World Data Integration
- **Data Sources:** World Bank API, US Export Controls API
- **Status:** {data_status}

## Country A: {selected_country_a}
- **GDP:** ${default_data[selected_country_a]['gdp']:.2f} Trillion
- **Military Expenditure:** {default_data[selected_country_a]['mil_exp']:.1f}% of GDP
- **Internet Penetration:** {default_data[selected_country_a]['internet']:.1f}%
- **Influence Score:** {default_data[selected_country_a]['influence']:.2f}
- **Policy Position:** {default_data[selected_country_a]['position']}

## Country B: {selected_country_b}
- **GDP:** ${default_data[selected_country_b]['gdp']:.2f} Trillion
- **Military Expenditure:** {default_data[selected_country_b]['mil_exp']:.1f}% of GDP
- **Internet Penetration:** {default_data[selected_country_b]['internet']:.1f}%
- **Influence Score:** {default_data[selected_country_b]['influence']:.2f}
- **Policy Position:** {default_data[selected_country_b]['position']}

## Adjudication Results
- **Geopolitical Tension:** {adjudication['tension_index']*100:.1f}%
- **Alignment Score:** {alignment_score*100:.0f}%
- **Adjudicator Confidence:** {adjudication['confidence_score']*100:.1f}%

## Deception Analysis
"""
        for actor, score in adjudication['deception_scores'].items():
            risk_pct = score * 100
            report += f"- **{actor}:** {risk_pct:.1f}% risk\\n"

        total_events = len(st.session_state['event_log'])
        report += f"""
## Latest Event
{adjudication['narrative']}

## Total Events: {total_events}
"""

        st.download_button(
            label="📄 Download Report (Markdown)",
            data=report,
            file_name=f"auracelle_report_round_{st.session_state['round']}.md",
            mime="text/markdown"
        )

with col_export2:
    if st.button("📋 Export Event Log"):
        if st.session_state["event_log"]:
            event_df = pd.DataFrame(st.session_state["event_log"])
            csv = event_df.to_csv(index=False)
            st.download_button(
                label="💾 Download CSV",
                data=csv,
                file_name=f"auracelle_events_round_{st.session_state['round']}.csv",
                mime="text/csv"
            )
        else:
            st.info("No events to export yet")

st.markdown("---")
st.caption("🤖 Auracelle Charlie Phase 2 - AI Agentic Adjudicator with Real-World Data Integration | Powered by World Bank, US Export Controls, and SIPRI")
'''

with open('pages/simulation.py', 'w') as f:
    f.write(sim_code)

print("✅ Full simulation page with API integration created")

# ========================================
# CELL 10: Launch Application
# ========================================
streamlit_process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "0.0.0.0"]
)
time.sleep(5)

ngrok_process = subprocess.Popen(
    ["ngrok", "http", "--domain=julene-untaxable-raelene.ngrok-free.dev", "8501"]
)

public_url = "https://aiwargame.ngrok.app"
print(f"\n🚀 Auracelle Charlie Phase 2 with Full API Integration is LIVE!")
print(f"🔗 Access at: {public_url}")
print(f"\n✨ Features:")
print("  • 🤖 AI Agentic Adjudicator")
print("  • 🌍 World Bank API (GDP, Military, Internet)")
print("  • 🚫 US Export Controls API (Sanctions)")
print("  • 💥 External Shock System")
print("  • 🎭 Deception Detection")
print("  • 📊 Real-World Data Dashboard")
print("  • 📈 Economic & Military Analysis")
print("  • 📥 Export Reports (Markdown & CSV)")
print(f"\n🔑 Password: charlie2025")
print(f"\n📊 Data Sources:")
print("  - World Bank: Automatic")
print("  - Export Controls: Automatic")
print("  - SIPRI: Upload CSV in sidebar")

display(HTML(f'<a href="{public_url}" target="_blank" style="font-size:20px; font-weight:bold; color:#0066cc;">🔗 Launch Auracelle Charlie - Live 2025</a>'))

^C
^C
✅ Setup complete
✅ AGPO Data Package created
✅ Enhanced Adjudicator with API integration created
✅ Login page created
✅ Full simulation page with API integration created

🚀 Auracelle Charlie Phase 2 with Full API Integration is LIVE!
🔗 Access at: https://aiwargame.ngrok.app

✨ Features:
  • 🤖 AI Agentic Adjudicator
  • 🌍 World Bank API (GDP, Military, Internet)
  • 🚫 US Export Controls API (Sanctions)
  • 💥 External Shock System
  • 🎭 Deception Detection
  • 📊 Real-World Data Dashboard
  • 📈 Economic & Military Analysis
  • 📥 Export Reports (Markdown & CSV)

🔑 Password: charlie2025

📊 Data Sources:
  - World Bank: Automatic
  - Export Controls: Automatic
  - SIPRI: Upload CSV in sidebar


In [ ]:

# === Launch Auracelle Charlie - Live 2025 Simulation ===
import subprocess
from pyngrok import ngrok
import time

# Kill old tunnels and servers
!pkill -f ngrok || true
!pkill -f streamlit || true
try:
    ngrok.kill()
    ngrok.disconnect()
except:
    pass

# Start Streamlit
process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0"])

# Start Ngrok with reserved domain
public_url = ngrok.connect(8501, hostname="aiwargame.ngrok.app")
print("✅ Tunnel created:", public_url)

from IPython.display import HTML, display
display(HTML(f'<a href="{public_url}" target="_blank" style="font-size:20px; font-weight:bold; color:#0066cc;">🔗 Launch Auracelle Charlie - Live 2025</a>'))


^C
^C
✅ Tunnel created: NgrokTunnel: "https://aiwargame.ngrok.app" -> "http://localhost:8501"


In [ ]:

# --- Add "Agentic AI Demo" Streamlit page ---
# This cell creates pages/90_Agentic_AI_Demo.py so it shows up as a separate page in the app.
# It is self-contained and will not modify your existing Simulation page or UI conventions.

import os
os.makedirs("pages", exist_ok=True)

agentic_page = r"""
# SPDX-License-Identifier: MIT
import time
import math
import random
from dataclasses import dataclass, field
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import streamlit as st

st.set_page_config(page_title="Agentic AI Demo", layout="wide")

# ------------------------------
# Minimal "aiwargame" sample env
# ------------------------------
# This is a lightweight, self-contained demonstration so it won't clash with your main Simulation logic.
# If your main app exposes shared policy/state helpers later, you can import them here and bypass this local demo.

@dataclass
class Actor:
    name: str
    influence: float = 1.0          # baseline influence (0-3 nominal range)
    compliance: float = 0.5         # nominal "policy compliance / safety posture" (0-1)
    payoff: float = 0.0             # accumulated reward

@dataclass
class Env:
    actors: Dict[str, Actor] = field(default_factory=dict)
    step_count: int = 0
    max_steps: int = 8
    history: List[Dict] = field(default_factory=list)

    def reset(self):
        self.step_count = 0
        self.history = []
        # lightweight baseline
        self.actors = {
            "US": Actor("US", influence=1.2, compliance=0.55),
            "EU": Actor("EU", influence=1.1, compliance=0.65),
            "China": Actor("China", influence=1.25, compliance=0.45),
        }
        return self._obs()

    def _obs(self):
        # Observation is a dict of actor states plus a derived "risk" score
        state = {
            k: {"influence": v.influence, "compliance": v.compliance, "payoff": v.payoff}
            for k, v in self.actors.items()
        }
        # simple aggregate "systemic risk" proxy
        mean_comp = np.mean([v.compliance for v in self.actors.values()])
        mean_inf = np.mean([v.influence for v in self.actors.values()])
        risk = float(max(0.0, 1.2 - (0.6*mean_comp + 0.2*min(1.5, mean_inf))))
        state["_derived"] = {"risk": risk, "t": self.step_count}
        return state

    def step(self, action: str, controlled_actor: str = "US"):
        # Apply an action taken by the "agent" controlling the selected actor.
        # Actions trade off compliance, influence, and a small coalition effect.
        a = self.actors[controlled_actor]

        # stochasticity for realism
        jitter = lambda s: s + random.uniform(-0.01, 0.01)

        coalition_bonus = 0.0
        if action == "Export Controls":
            a.influence = max(0.6, jitter(a.influence + 0.02))
            a.compliance = min(1.0, jitter(a.compliance + 0.04))
        elif action == "Safety Audits":
            a.influence = max(0.6, jitter(a.influence - 0.01))
            a.compliance = min(1.0, jitter(a.compliance + 0.06))
            coalition_bonus = 0.01
        elif action == "Open Data":
            a.influence = min(1.6, jitter(a.influence + 0.03))
            a.compliance = max(0.1, jitter(a.compliance - 0.02))
        elif action == "Joint Standards":
            a.influence = min(1.6, jitter(a.influence + 0.02))
            a.compliance = min(1.0, jitter(a.compliance + 0.03))
            coalition_bonus = 0.02
        else:
            # No-op / hold
            a.influence = jitter(a.influence)
            a.compliance = jitter(a.compliance)

        # Simple coalition effect: others follow a tiny bit
        for k, other in self.actors.items():
            if k == controlled_actor:
                continue
            other.compliance = float(np.clip(other.compliance + coalition_bonus * random.uniform(0.4, 1.2), 0.1, 1.0))

        # Reward = (own influence * 0.6 + own compliance * 0.8) - systemic risk penalty
        obs = self._obs()
        risk = obs["_derived"]["risk"]
        reward = 0.6*a.influence + 0.8*a.compliance - (0.9 * risk)
        a.payoff += reward

        self.step_count += 1
        done = self.step_count >= self.max_steps

        # Log
        self.history.append({
            "t": self.step_count,
            "actor": controlled_actor,
            "action": action,
            "reward": reward,
            "risk": risk,
            "US_infl": self.actors["US"].influence,
            "US_comp": self.actors["US"].compliance,
            "EU_infl": self.actors["EU"].influence,
            "EU_comp": self.actors["EU"].compliance,
            "CN_infl": self.actors["China"].influence,
            "CN_comp": self.actors["China"].compliance,
        })

        return obs, reward, done, {}

# ------------------------------
# A tiny "Agentic AI" controller
# ------------------------------
# A one-step lookahead agent that evaluates each action with a heuristic objective
# and chooses the best. This is intentionally transparent for demo purposes.

ACTIONS = ["Export Controls", "Safety Audits", "Open Data", "Joint Standards", "Hold/No-Op"]

def evaluate_action(env: Env, action: str, actor: str, sims: int = 12):
    # simulate hypothetical outcome for ranking; keep it light-weight
    # Copy a shallow snapshot for Monte Carlo trials
    scores = []
    for _ in range(sims):
        snap = Env()
        # clone state
        snap.actors = {k: Actor(v.name, v.influence, v.compliance, v.payoff) for k, v in env.actors.items()}
        snap.step_count = env.step_count
        obs, reward, _, _ = snap.step(action, controlled_actor=actor)
        # Heuristic: prefer high own reward, lower systemic risk
        srisk = obs["_derived"]["risk"]
        score = reward - 0.3*srisk
        scores.append(score)
    return float(np.mean(scores))

def agent_choose_action(env: Env, actor: str, stochastic: bool = False):
    candidates = []
    for a in ACTIONS:
        s = evaluate_action(env, a, actor)
        candidates.append((a, s))
    candidates.sort(key=lambda x: x[1], reverse=True)
    if stochastic and random.random() < 0.2:
        return random.choice(ACTIONS)
    return candidates[0][0]

# ------------------------------
# UI
# ------------------------------

st.title("🤖 Agentic AI Demo — Sample Game")
st.caption("A transparent, one-step lookahead agent plays a lightweight version of the AI governance wargame.")

left, right = st.columns([2, 1])

with right:
    st.subheader("Agent Controls")
    controlled_actor = st.selectbox("Controlled Actor", ["US", "EU", "China"], index=0)
    horizon = st.slider("Episode Length (steps)", 4, 16, 8, 1)
    stochastic = st.toggle("Enable stochastic exploration (ε≈0.2)", value=True)
    episodes = st.number_input("Batch episodes", min_value=1, max_value=50, value=1, step=1)
    autoplay = st.toggle("Autoplay (fast)", value=False)
    st.markdown("---")
    st.page_link("app.py", label="⬅ Back to Home", icon="🏠")

with left:
    st.subheader("Run a Sample Game")
    env = Env()
    env.max_steps = horizon

    run_now = st.button("▶ Play Sample Game")

    if run_now:
        obs = env.reset()
        frames = []
        for t in range(horizon):
            action = agent_choose_action(env, controlled_actor, stochastic=stochastic)
            obs, reward, done, _ = env.step(action, controlled_actor)
            row = {"t": t+1, "action": action, "reward": round(reward, 4), "risk": round(obs["_derived"]["risk"], 4)}
            for k, v in env.actors.items():
                row[f"{k}_influence"] = round(v.influence, 3)
                row[f"{k}_compliance"] = round(v.compliance, 3)
                row[f"{k}_payoff"] = round(v.payoff, 3)
            frames.append(row)
            if not autoplay:
                st.write(f"Step {t+1}: **{action}** → reward {row['reward']}, risk {row['risk']}")
                time.sleep(0.25)
            if done:
                break

        df = pd.DataFrame(frames)
        st.markdown("#### Episode Trace")
        st.dataframe(df, use_container_width=True)

        # Simple summaries
        st.markdown("#### Final Payoffs")
        summary = pd.DataFrame([{"Actor": k, "Payoff": round(v.payoff, 4), "Influence": round(v.influence,3), "Compliance": round(v.compliance,3)}
                                for k, v in env.actors.items()]).sort_values("Payoff", ascending=False)
        st.dataframe(summary, use_container_width=True)

    st.markdown("---")
    st.subheader("Batch Evaluation")
    if st.button("🏃 Run Batch"):
        results = []
        for ep in range(episodes):
            env = Env(); env.max_steps = horizon; env.reset()
            for t in range(horizon):
                a = agent_choose_action(env, controlled_actor, stochastic=stochastic)
                env.step(a, controlled_actor)
            results.append({k: v.payoff for k, v in env.actors.items()})
        res_df = pd.DataFrame(results)
        st.markdown("Average payoff over batch:")
        st.dataframe(pd.DataFrame(res_df.mean()).rename(columns={0:"Avg Payoff"}), use_container_width=True)

"""

with open("pages/90_Agentic_AI_Demo.py", "w", encoding="utf-8") as f:
    f.write(agentic_page)

print("✅ Created pages/90_Agentic_AI_Demo.py")


✅ Created pages/90_Agentic_AI_Demo.py


In [ ]:

# --- Write auracelle_agent_adapter.py (correctly) ---
adapter_src = r"""
import importlib
import random
from typing import List
import numpy as np
import streamlit as st

def _try_get(module, names: List[str]):
    for n in names:
        if hasattr(module, n):
            return getattr(module, n)
    return None

def _coerce_list(x):
    try:
        return list(x)
    except Exception:
        return None

def load_main_sim_handles():
    try:
        app = importlib.import_module("app")
    except Exception:
        return None
    handles = {}
    handles["policy_options"] = _try_get(app, ["policy_options","POLICY_OPTIONS","policies","POLICY_LIST"])
    handles["policy_effects"] = _try_get(app, ["policy_effect_mappings","policy_effects","POLICY_EFFECTS","policy_effect_map"])
    handles["countries"] = _try_get(app, ["countries","country_list","COUNTRIES","nodes","NODES"])
    handles["roles"] = _try_get(app, ["roles","ROLES"])
    handles["apply_policy"] = _try_get(app, ["apply_policy_effects","apply_policy","apply_effects"])
    handles["get_risk"] = _try_get(app, ["compute_systemic_risk","get_risk","risk_metric"])
    for k in ["policy_options","countries","roles"]:
        if handles.get(k) is not None:
            handles[k] = _coerce_list(handles[k])
    handles["app"] = app
    return handles

def init_state(actors: List[str]):
    if "agent_autoplay_state" not in st.session_state:
        st.session_state.agent_autoplay_state = {
            "t": 0,
            "actors": {a: {"influence": 1.0, "compliance": 0.5, "payoff": 0.0} for a in actors},
            "history": []
        }
    return st.session_state.agent_autoplay_state

def toy_risk(state: dict):
    inf = np.mean([v["influence"] for v in state["actors"].values()])
    comp = np.mean([v["compliance"] for v in state["actors"].values()])
    return float(max(0.0, 1.2 - (0.6*comp + 0.2*min(1.5, inf))))

def derive_risk(handles, state):
    if handles and handles.get("get_risk"):
        try:
            return float(handles["get_risk"](state))
        except Exception:
            pass
    return toy_risk(state)

def step_with_main_effects(handles, state: dict, action: str, controlled_actor: str):
    if handles and handles.get("apply_policy"):
        try:
            new_state = handles["apply_policy"](state, action, controlled_actor)
            st.session_state.agent_autoplay_state = new_state
            risk = derive_risk(handles, new_state)
            a = new_state["actors"][controlled_actor]
            reward = 0.6*a["influence"] + 0.8*a["compliance"] - 0.9*risk
            a["payoff"] += reward
            new_state["t"] = new_state.get("t", 0) + 1
            return reward, risk
        except Exception:
            pass
    pe = None
    if handles and handles.get("policy_effects"):
        pe = handles["policy_effects"]
    if isinstance(pe, dict) and action in pe:
        delta = pe[action]
        a = state["actors"].setdefault(controlled_actor, {"influence": 1.0, "compliance": 0.5, "payoff": 0.0})
        if isinstance(delta, dict):
            if "influence" in delta:
                a["influence"] = float(np.clip(a["influence"] + float(delta["influence"]), 0.1, 2.0))
            if "compliance" in delta:
                a["compliance"] = float(np.clip(a["compliance"] + float(delta["compliance"]), 0.0, 1.0))
        risk = derive_risk(handles, state)
        reward = 0.6*a["influence"] + 0.8*a["compliance"] - 0.9*risk
        a["payoff"] += reward
        state["t"] += 1
        return reward, risk
    # fallback toy step
    a = state["actors"].setdefault(controlled_actor, {"influence": 1.0, "compliance": 0.5, "payoff": 0.0})
    jitter = lambda s: s + random.uniform(-0.01, 0.01)
    if action == "Export Controls":
        a["influence"] = max(0.6, jitter(a["influence"] + 0.02))
        a["compliance"] = min(1.0, jitter(a["compliance"] + 0.04))
    elif action == "Safety Audits":
        a["influence"] = max(0.6, jitter(a["influence"] - 0.01))
        a["compliance"] = min(1.0, jitter(a["compliance"] + 0.06))
    elif action == "Open Data":
        a["influence"] = min(1.6, jitter(a["influence"] + 0.03))
        a["compliance"] = max(0.1, jitter(a["compliance"] - 0.02))
    elif action == "Joint Standards":
        a["influence"] = min(1.6, jitter(a["influence"] + 0.02))
        a["compliance"] = min(1.0, jitter(a["compliance"] + 0.03))
    else:
        a["influence"] = jitter(a["influence"])
        a["compliance"] = jitter(a["compliance"])
    risk = derive_risk(handles, state)
    reward = 0.6*a["influence"] + 0.8*a["compliance"] - 0.9*risk
    a["payoff"] += reward
    state["t"] += 1
    return reward, risk

def get_actions(handles):
    if handles and handles.get("policy_options"):
        return [str(x) for x in handles["policy_options"]]
    return ["Export Controls","Safety Audits","Open Data","Joint Standards","Hold/No-Op"]

def get_actors(handles):
    if handles and handles.get("countries"):
        return [str(x) for x in handles["countries"]]
    return ["US","EU","China"]

def evaluate_action(handles, state, action: str, actor: str, sims: int = 8):
    import copy
    scores = []
    for _ in range(sims):
        s = copy.deepcopy(state)
        reward, risk = step_with_main_effects(handles, s, action, actor)
        scores.append(reward - 0.3*risk)
    return float(np.mean(scores))

def agent_choose_action(handles, state, actor: str, stochastic: bool = False):
    acts = get_actions(handles)
    ranked = [(a, evaluate_action(handles, state, a, actor)) for a in acts]
    ranked.sort(key=lambda x: x[1], reverse=True)
    if stochastic and random.random() < 0.2:
        return random.choice(acts)
    return ranked[0][0]
"""
with open("auracelle_agent_adapter.py", "w", encoding="utf-8") as f:
    f.write(adapter_src)
print("✅ Wrote auracelle_agent_adapter.py")


✅ Wrote auracelle_agent_adapter.py


In [ ]:

# --- Write pages/70_Agent_Autoplay_MainSim.py ---
import os
os.makedirs("pages", exist_ok=True)

page_src = r"""
import time
import pandas as pd
import streamlit as st
from auracelle_agent_adapter import (
    load_main_sim_handles, init_state, get_actions, get_actors,
    agent_choose_action, step_with_main_effects
)
st.set_page_config(page_title='Agent Autoplay (Main Simulation)', layout='wide')
st.title('🤖 Agent Autoplay — Main Simulation Policies')
st.caption("Uses your main simulation's policy options/effects when available. Falls back to a safe demo if not.")

handles = load_main_sim_handles()
actors = get_actors(handles)
state = init_state(actors)

left, right = st.columns([2, 1])

with right:
    st.subheader('Agent Controls')
    controlled_actor = st.selectbox('Controlled Actor', actors, index=0)
    horizon = st.slider('Episode Length (steps)', 4, 20, 8, 1)
    stochastic = st.toggle('Enable stochastic exploration (ε≈0.2)', value=True)
    episodes = st.number_input('Batch episodes', min_value=1, max_value=100, value=1, step=1)
    autoplay = st.toggle('Autoplay (fast)', value=False)
    st.markdown('---')
    st.page_link('app.py', label='⬅ Back to Simulation', icon='🏠')

with left:
    st.subheader('Run a Sample Game (Main Sim)')
    if st.button('▶ Play Episode'):
        st.session_state.agent_autoplay_state = {
            't': 0,
            'actors': {a: {'influence': 1.0, 'compliance': 0.5, 'payoff': 0.0} for a in actors},
            'history': []
        }
        state = st.session_state.agent_autoplay_state

        frames = []
        for t in range(horizon):
            action = agent_choose_action(handles, state, controlled_actor, stochastic=stochastic)
            reward, risk = step_with_main_effects(handles, state, action, controlled_actor)
            row = {'t': t+1, 'action': action, 'reward': round(reward,4), 'risk': round(risk,4)}
            for k, v in state['actors'].items():
                row[f'{k}_influence'] = round(v['influence'],3)
                row[f'{k}_compliance'] = round(v['compliance'],3)
                row[f'{k}_payoff'] = round(v['payoff'],3)
            frames.append(row)
            if not autoplay:
                st.write(f"Step {t+1}: **{action}** → reward {row['reward']}, risk {row['risk']}")
                time.sleep(0.25)

        df = pd.DataFrame(frames)
        st.markdown('#### Episode Trace')
        st.dataframe(df, use_container_width=True)

        st.markdown('#### Final Payoffs')
        summary = pd.DataFrame([{'Actor': k, 'Payoff': round(v['payoff'],4), 'Influence': round(v['influence'],3), 'Compliance': round(v['compliance'],3)}
                                for k, v in state['actors'].items()]).sort_values('Payoff', ascending=False)
        st.dataframe(summary, use_container_width=True)

    st.markdown('---')
    st.subheader('Batch Evaluation')
    if st.button('🏃 Run Batch'):
        results = []
        for ep in range(episodes):
            st.session_state.agent_autoplay_state = {
                't': 0,
                'actors': {a: {'influence': 1.0, 'compliance': 0.5, 'payoff': 0.0} for a in actors},
                'history': []
            }
            s = st.session_state.agent_autoplay_state
            for t in range(horizon):
                a = agent_choose_action(handles, s, controlled_actor, stochastic=stochastic)
                step_with_main_effects(handles, s, a, controlled_actor)
            results.append({k: v['payoff'] for k, v in s['actors'].items()})
        res_df = pd.DataFrame(results)
        st.markdown('Average payoff over batch:')
        st.dataframe(pd.DataFrame(res_df.mean()).rename(columns={0:'Avg Payoff'}), use_container_width=True)
"""
with open("pages/70_Agent_Autoplay_MainSim.py", "w", encoding="utf-8") as f:
    f.write(page_src)
print("✅ Wrote pages/70_Agent_Autoplay_MainSim.py")


✅ Wrote pages/70_Agent_Autoplay_MainSim.py


In [ ]:

# --- Optional: inject sidebar link into app.py ---
import os
if os.path.exists("app.py"):
    with open("app.py", "r", encoding="utf-8") as f:
        content = f.read()
    if "AGENT_AUTOPLAY_LINK_INJECTED" not in content:
        content += "\n\n# AGENT_AUTOPLAY_LINK_INJECTED\ntry:\n    import streamlit as st\n    st.sidebar.page_link('pages/70_Agent_Autoplay_MainSim.py', label='🤖 Agent Autoplay (Main Sim)')\nexcept Exception:\n    pass\n"
        with open("app.py", "w", encoding="utf-8") as f:
            f.write(content)
        print("✅ Injected sidebar link into app.py")
    else:
        print("ℹ️ Sidebar link already present")
else:
    print("ℹ️ app.py not found now; after your setup cell writes it, re-run this cell to inject the link.")


✅ Injected sidebar link into app.py
